## Simulating Airplane Boarding Strategies

### Inspiration:
* Video by CPG Grey: https://www.youtube.com/watch?v=oAHbLRjF0vo&feature=youtu.be
* This video borrows from work by Jason Stefen, whose paper is here: https://arxiv.org/pdf/0802.0733.pdf

### What does this notebook do?

Given a boarding policy (that is, a rule which assigns a boarding order to passengers on a plane), simulate the amount of time it'll take to board the plane using that policy. Example policies include boarding front to back, back to front, and window-middle-aisle.

The amount of time it takes each passenger to put their luggage away and sit down is distributed ~ N(mu, sigma), where mu and sigma are defined in the Plane object.

To create a boarding policy, subclass `AbstractBoardingPolicy` and fill in the abstract methods.

Then instantiate a `PlaneBoardingSimulator` and off you go:

```
planeInitParams = {
    "numRows": 20, 
    "numSeatsPerRow": 6
}

policies = [
    FrontRowsFirstBoardingPolicy(), 
    BackRowsFirstBoardingPolicy(), 
    WindowMiddleAisleBoardingPolicy()
]

pbs = PlaneBoardingSimulator(planeInitParams)

# plots a histogram
pbs.plotPolicyHistogram(policies, iterationsPerPolicy=50)

or

# list of boarding times, one time per iteration
rawTimes = pbs.simulatePolicy(WindowMiddleAisleBoardingPolicy())

```

In [30]:
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from multiprocessing.pool import ThreadPool

In [31]:
class Passenger():
    AVG_GETTING_SEATED_TIME_IN_SECONDS = 5
    STDDEV_GETTING_SEATED_TIME_IN_SECONDS = 3
    MINIMUM_POSSIBLE_BOARDING_TIME_IN_SECONDS = 2
    
    def __init__(self, rowNumber, seatNumber, pos = None):
        self.id = None
        self.rowNumber = rowNumber
        self.seatNumber = seatNumber
        self.pos = pos
        self.hasBoarded = False
        self.boardingStatus = "waiting"
        self.timeUntilSeated = None
        self.waitingTime = 0
        
    def sampleGettingSeatedTimeDistribution(self):
        sample = np.random.normal(
            Passenger.AVG_GETTING_SEATED_TIME_IN_SECONDS,
            Passenger.STDDEV_GETTING_SEATED_TIME_IN_SECONDS,
            1)[0]
        
        return max(sample, Passenger.MINIMUM_POSSIBLE_BOARDING_TIME_IN_SECONDS)
            
    def setPos(self, pos):
        self.pos = pos
        
    def setTimeUntilSeated(self, timeUntilSeated):
        self.timeUntilSeated = timeUntilSeated
        
    def setBoardingStatus(self, boardingStatus):
        self.boardingStatus = boardingStatus
        
    def incWaitingTime(self, t):
        self.waitingTime += t
        
    def setId(self, _id):
        self.id = _id
        
    def __str__(self):
        return "Row: {}, Seat: {}, hasBoarded: {}, pos: {}, waitingTime: {}, walkingTimeToRow: {}"\
            .format(self.rowNumber, self.seatNumber, self.hasBoarded, 
                    self.pos, self.waitingTime, self.walkingTimeToRow)

In [32]:
class Plane():
    WALKING_TIME_TO_NEXT_ROW_IN_SECONDS = 2
    
    def __init__(self, numRows, numSeatsPerRow):
        self.numRows = numRows
        self.rowOccupiedLookup = {rowNum: False for rowNum in range(numRows)}
        self.numSeatsPerRow = numSeatsPerRow
        self.totalNumPassengers = numRows * numSeatsPerRow
        self.numPassengersBoardedSoFar = 0
        self.passengers = None
        
    def board(self, boardingPolicy):
        self.numPassengersBoardedSoFar = 0
        self.passengers = self._genPassengersViaPolicy(boardingPolicy)
        self._movePassengersForwardUntilDone()
        totalBoardingTimeInMinutes = self._getMaxWaitingTime() / 60
        return totalBoardingTimeInMinutes
    
    def _getMaxWaitingTime(self):
        return max(list(map(lambda p: p. waitingTime, self.passengers)))
    
    def _setBoardingStatusForPassenger(self, p, boardingStatus):
        p.setBoardingStatus(boardingStatus)
        
        if boardingStatus == "seated":
            self.numPassengersBoardedSoFar += 1
            assert(p.pos == p.rowNumber)
            self.rowOccupiedLookup[p.pos] = False
        elif boardingStatus == "inProgress":
            self.rowOccupiedLookup[p.pos] = True
    
    def _movePassengersForwardUntilDone(self):
        while self._somePassengersStillNeedToBoard():
            for p in self.passengers:
                if p.boardingStatus == "inProgress":
                    p.setTimeUntilSeated(p.timeUntilSeated - 1)
                    p.incWaitingTime(1)
                    if p.timeUntilSeated <= 0:
                        self._setBoardingStatusForPassenger(p, "seated")
                elif p.boardingStatus == "waiting":
                    if p.pos < p.rowNumber and self.rowOccupiedLookup[p.pos + 1] == False:
                        self.rowOccupiedLookup[p.pos] = False
                        p.setPos(p.pos + 1)
                        self.rowOccupiedLookup[p.pos] = True
                        p.incWaitingTime(Plane.WALKING_TIME_TO_NEXT_ROW_IN_SECONDS)
                        if p.pos == p.rowNumber:
                            self._setBoardingStatusForPassenger(p, "inProgress")
                            gettingSeatedTime = p.sampleGettingSeatedTimeDistribution()
                            p.setTimeUntilSeated(gettingSeatedTime)
                    elif self.rowOccupiedLookup[p.pos + 1] == True:
                        # can't move yet
                        p.incWaitingTime(1)
                    else:
                        raise Exception("Bad state: {}".format(p))
                elif p.boardingStatus == "seated":
                    continue
                else:
                    raise Exception("Boarding status invalid: {}".format(p.boardingStatus))
    
    def _somePassengersStillNeedToBoard(self):
        return self.numPassengersBoardedSoFar < self.totalNumPassengers
    
    def _getBoardablePassengerIds(self, passengers):
        idxs = []
        for idx, p in enumerate(passengers):
            if (p.rowNumber == p.pos) and (p.hasBoarded == False):
                idxs.append(idx)
        return idxs
    
    def _genPassengersViaPolicy(self, boardingPolicy):
        unorderedPassengers = [
            Passenger(rowNumber, seatNumber)
            for rowNumber in range(self.numRows)
            for seatNumber in range(self.numSeatsPerRow)
        ]
        
        orderedPassengers = boardingPolicy.orderPassengers(unorderedPassengers)
        
        for idx, p in enumerate(orderedPassengers):
            p.setId(idx) # will never change
            p.setPos(-1 * idx - 1)
        
        return orderedPassengers

In [33]:
class AbstractBoardingPolicy(ABC):    
    @abstractmethod
    def orderPassengers(self, passengers):
        pass
    def name(self):
        pass

class BackRowsFirstBoardingPolicy(AbstractBoardingPolicy):
    def orderPassengers(self, passengers):
        return sorted(passengers, key = lambda passenger: -1 * passenger.rowNumber)
    def name(self):
        return "Back Rows First"

class FrontRowsFirstBoardingPolicy(AbstractBoardingPolicy):
    def orderPassengers(self, passengers):
        return sorted(passengers, key = lambda passenger: passenger.rowNumber)
    def name(self):
        return "Front Rows First"
    
class WindowMiddleAisleBoardingPolicy(AbstractBoardingPolicy):
    """
    a smart boarding strategy maximizes parallel boarding (thereby minimizing time bottlenecks).
    this strategy minimizes seat shuffles (when person A has to get out of her row so person B can get into it)
    """
    def orderPassengers(self, passengers):
        return sorted(passengers, key = lambda passenger: (passenger.seatNumber, -1 * passenger.rowNumber))
    def name(self):
        return "Window Middle Aisle"

In [34]:
# spot check to ensure policies are doing what we want them to
policies = [
    FrontRowsFirstBoardingPolicy(), 
    BackRowsFirstBoardingPolicy(), 
    WindowMiddleAisleBoardingPolicy()
]

for policy in policies:
    print("=========Policy: {}=========".format(policy.name()))
    plane = Plane(numRows = 20, numSeatsPerRow = 6)
    passengers = plane._genPassengersViaPolicy(policy)
    orderedPassengers = list(
        map(
            lambda x: "Row: {}, Seat: {}".format(x.rowNumber, x.seatNumber), passengers
        )
    )
    print("======First 5 passengers:====== \n{}".format(orderedPassengers[:5]))
    print("======Last 5 passengers:====== \n{}\n".format(orderedPassengers[-5:]))

=========Policy: Front Rows First=========
======First 5 passengers:====== 
['Row: 0, Seat: 0', 'Row: 0, Seat: 1', 'Row: 0, Seat: 2', 'Row: 0, Seat: 3', 'Row: 0, Seat: 4']
======Last 5 passengers:====== 
['Row: 19, Seat: 1', 'Row: 19, Seat: 2', 'Row: 19, Seat: 3', 'Row: 19, Seat: 4', 'Row: 19, Seat: 5']

=========Policy: Back Rows First=========
======First 5 passengers:====== 
['Row: 19, Seat: 0', 'Row: 19, Seat: 1', 'Row: 19, Seat: 2', 'Row: 19, Seat: 3', 'Row: 19, Seat: 4']
======Last 5 passengers:====== 
['Row: 0, Seat: 1', 'Row: 0, Seat: 2', 'Row: 0, Seat: 3', 'Row: 0, Seat: 4', 'Row: 0, Seat: 5']

=========Policy: Window Middle Aisle=========
======First 5 passengers:====== 
['Row: 19, Seat: 0', 'Row: 18, Seat: 0', 'Row: 17, Seat: 0', 'Row: 16, Seat: 0', 'Row: 15, Seat: 0']
======Last 5 passengers:====== 
['Row: 4, Seat: 5', 'Row: 3, Seat: 5', 'Row: 2, Seat: 5', 'Row: 1, Seat: 5', 'Row: 0, Seat: 5']



In [37]:
class PlaneBoardingSimulator():
    def __init__(self, planeKwargs):
        self.planeKwargs = planeKwargs
        self.totalNumPassengers = planeKwargs["numRows"] * planeKwargs["numSeatsPerRow"]
    
    def generatePlane(self):
        return Plane(**self.planeKwargs)
    
    def _simulatePolicySynchronous(self, policy):
        plane = self.generatePlane()
        return plane.board(policy)
    
    def simulatePolicy(self, policy, n=100):
        pool = ThreadPool(16)
        output = pool.map(self._simulatePolicySynchronous, [policy]*n)
        pool.close() 
        pool.join() 
        return output
    
    def plotPolicyHistogram(self, policies, iterationsPerPolicy=100):
        plt.figure(figsize = (16, 8))
        plt.title("Simulated Boarding Times Based on Various Strategies (Total # Passengers: {})".format(self.totalNumPassengers), fontsize=20)
        plt.ylabel("Count", fontsize=18)
        plt.xlabel("Total Boarding Time (minutes)", fontsize=18)
        plt.gca().set_facecolor("honeydew")
        colors = ["red", "blue", "orange", "green", "purple"]

        for policy in policies:
            print("Computing times for policy: {}".format(policy.name()))
            color = np.random.choice(colors)
            times = self.simulatePolicy(policy, n=iterationsPerPolicy)
            _ = plt.hist(times, color=color, alpha=.9)
            mu = np.round(np.mean(times), 1)
            sigma = np.round(np.std(times), 1)
            plt.axvline(mu, c=color, 
                        label="{} (μ = {}, σ = {})".format(policy.name(), mu, sigma))
            plt.legend(fontsize = 16)
            plt.gca().tick_params(labelsize=16)
        
        plt.show()

In [ ]:
planeInitParams = {
    "numRows": 50, 
    "numSeatsPerRow": 6
}

policies = [
    FrontRowsFirstBoardingPolicy(), 
    BackRowsFirstBoardingPolicy(), 
    WindowMiddleAisleBoardingPolicy()
]

PlaneBoardingSimulator(
    planeInitParams,
).plotPolicyHistogram(policies, iterationsPerPolicy=50)

Computing times for policy: Front Rows First
Computing times for policy: Back Rows First
